# 모듈 4: ADK에서 구조화된 출력(Structured Outputs) 사용하기

이 모듈에서는 ADK의 출력 스키마 기능을 사용하여 에이전트가 일관되고 구조화된 형식(JSON)으로 데이터를 반환하도록 강제하는 방법을 배운다.

## 학습 목표

이 모듈을 완료하면 다음 내용을 이해할 수 있다.

- **Pydantic 활용**: Python의 데이터 검증 라이브러리인 Pydantic을 사용하여 데이터 구조를 정의한다.
- **구조화된 응답 강제**: 에이전트가 정의된 스키마에 맞춰 정확한 JSON 데이터를 생성하도록 만든다.
- **타입 검증**: Enum과 Optional 등을 사용하여 정교한 데이터 유효성 검사를 수행한다.

## 구조화된 출력이 필요한 이유

AI 에이전트는 기본적으로 자유로운 형식의 자연어로 응답한다. 하지만 에이전트의 응답을 웹사이트에 표시하거나 데이터베이스에 저장해야 한다면 자연어는 처리하기 어렵다. 이때 **구조화된 출력**이 필요하다.

- **일관성**: 항상 약속된 필드와 포맷(JSON)으로 데이터를 받을 수 있다.
- **자동화**: 별도의 파싱 로직 없이 바로 코드에서 객체로 사용할 수 있다.
- **오류 감소**: 필수 정보가 누락되는 것을 방지한다.

## 환경 설정

데이터 구조 정의를 위해 `pydantic` 라이브러리가 필요하다. 아래 코드로 설치한다.

In [ ]:
# 필요한 패키지 설치
%pip install -q google-adk python-dotenv pydantic

In [1]:
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

MODEL = "gemini-2.5-flash"

True

## 예제 1: 이메일 생성 에이전트

첫 번째 예제에서는 제목(subject)과 본문(body)으로 분리된 구조화된 이메일 콘텐츠를 생성하는 에이전트를 만든다.

### Pydantic 모델 정의하기

`BaseModel`을 상속받아 우리가 원하는 데이터의 '설계도'를 만든다. 이 클래스는 에이전트에게 "반드시 이 형태로 대답해"라고 알려주는 역할을 한다.

In [2]:
from pydantic import BaseModel, Field

# 이메일 구조 정의 (Pydantic)
class EmailContent(BaseModel):
    subject: str = Field(description="이메일의 제목")
    body: str = Field(description="이메일의 본문 내용")

### 이메일 에이전트 생성하기

`output_type` (또는 `output_schema`) 파라미터에 위에서 정의한 Pydantic 클래스를 전달한다. 또한 `instructions`에 한국어 프롬프트를 적용하여 에이전트가 한국어로 이메일을 작성하도록 한다.

In [9]:
from google.adk import Agent
from google.adk.runners import InMemoryRunner

# Agent 정의
email_agent = Agent(
    name="email_agent",
    model=MODEL,
    description="비즈니스 이메일 작성 어시스턴트",
    instruction="""
    당신은 전문적인 비즈니스 이메일 작성 비서입니다.
    사용자의 요청에 따라 정중하고 명확한 이메일을 작성하세요.
    
    작성 지침:
    1. 간결하고 핵심이 담긴 제목을 작성하세요.
    2. 인사말, 본문, 맺음말을 갖춘 격식 있는 본문을 작성하세요.
    3. 항상 비즈니스 매너를 지키세요.
    """,
    output_schema=EmailContent
)

# Runner 생성
email_runner = InMemoryRunner(agent=email_agent)

# 이메일 작성 요청
await email_runner.run_debug("프로젝트 마감일 연장에 대해 팀원들에게 알리는 이메일을 써줘.")


 ### Created new session: debug_session_id

User > 프로젝트 마감일 연장에 대해 팀원들에게 알리는 이메일을 써줘.
email_agent > {
  "subject": "프로젝트 마감일 연장 안내",
  "body": "팀원 여러분께,\n\n현재 진행 중인 프로젝트의 마감일이 연장되었음을 알려드립니다.\n\n변경된 마감일은 [새로운 마감일]입니다. 이번 연장은 [연장 사유]로 인해 결정되었습니다. 팀원 여러분의 노고에 감사드리며, 이번 연장이 프로젝트를 성공적으로 완료하는 데 도움이 되기를 바랍니다.\n\n새로운 마감일에 맞춰 업무 계획을 재조정해주시고, 궁금한 점이 있으시면 언제든지 저에게 문의해주세요.\n\n감사합니다.\n\n[이름] 드림"
}


[Event(model_version='gemini-2.0-flash', content=Content(
   parts=[
     Part(
       text="""{
   "subject": "프로젝트 마감일 연장 안내",
   "body": "팀원 여러분께,\n\n현재 진행 중인 프로젝트의 마감일이 연장되었음을 알려드립니다.\n\n변경된 마감일은 [새로운 마감일]입니다. 이번 연장은 [연장 사유]로 인해 결정되었습니다. 팀원 여러분의 노고에 감사드리며, 이번 연장이 프로젝트를 성공적으로 완료하는 데 도움이 되기를 바랍니다.\n\n새로운 마감일에 맞춰 업무 계획을 재조정해주시고, 궁금한 점이 있으시면 언제든지 저에게 문의해주세요.\n\n감사합니다.\n\n[이름] 드림"
 }"""
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   candidates_token_count=198,
   candidates_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=198
     ),
   ],
   prompt_token_count=202,
   prompt_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=202
     ),
   ],
   total_toke

### 실행 및 결과 확인

`run_debug`를 실행하면 에이전트가 내부적으로 JSON 구조를 생성하는 것을 볼 수 있다. ADK는 모델의 출력을 자동으로 파싱하여 `EmailContent` 객체로 변환한다.

In [10]:
# 이메일 작성 요청
await email_runner.run_debug("프로젝트 마감일 연장에 대해 팀원들에게 알리는 이메일을 써줘.")


 ### Continue session: debug_session_id

User > 프로젝트 마감일 연장에 대해 팀원들에게 알리는 이메일을 써줘.
email_agent > {
  "subject": "프로젝트 마감일 연장 안내",
  "body": "팀원 여러분께,\n\n현재 진행 중인 프로젝트의 마감일이 연장되었음을 알려드립니다. 변경된 마감일은 2024년 5월 31일입니다. 이번 연장은 예기치 않은 기술적인 문제로 인해 결정되었습니다. 팀원 여러분의 노고에 감사드리며, 이번 연장이 프로젝트를 성공적으로 완료하는 데 도움이 되기를 바랍니다.\n\n새로운 마감일에 맞춰 업무 계획을 재조정해주시고, 궁금한 점이 있으시면 언제든지 저에게 문의해주세요.\n\n감사합니다.\n\n김민수 드림"
}


[Event(model_version='gemini-2.0-flash', content=Content(
   parts=[
     Part(
       text="""{
   "subject": "프로젝트 마감일 연장 안내",
   "body": "팀원 여러분께,\n\n현재 진행 중인 프로젝트의 마감일이 연장되었음을 알려드립니다. 변경된 마감일은 2024년 5월 31일입니다. 이번 연장은 예기치 않은 기술적인 문제로 인해 결정되었습니다. 팀원 여러분의 노고에 감사드리며, 이번 연장이 프로젝트를 성공적으로 완료하는 데 도움이 되기를 바랍니다.\n\n새로운 마감일에 맞춰 업무 계획을 재조정해주시고, 궁금한 점이 있으시면 언제든지 저에게 문의해주세요.\n\n감사합니다.\n\n김민수 드림"
 }"""
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   candidates_token_count=201,
   candidates_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=201
     ),
   ],
   prompt_token_count=425,
   prompt_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=425
     ),
   ],
   tota

## 예제 2: 복잡한 구조 (지원 티켓 시스템)

이번에는 더 정교한 제어가 필요한 '고객 지원 티켓' 생성 에이전트를 만든다. 여기서는 `Enum`(열거형)과 `Optional`(선택 항목)을 사용하여 데이터의 유효성을 더욱 엄격하게 관리한다.

### 고급 Pydantic 모델 정의

- **Enum**: `priority` 필드는 미리 정해진 값(Low, Medium, High, Critical) 중 하나만 가질 수 있다.
- **List**: `steps` 필드는 문자열의 리스트(배열) 형태여야 한다.
- **Optional**: `contact` 필드는 없어도 에러가 나지 않는다.

In [11]:
from typing import List, Optional
from enum import Enum

# 우선순위 (선택지 제한)
class Priority(str, Enum):
    LOW = "낮음"
    MEDIUM = "보통"
    HIGH = "높음"
    CRITICAL = "긴급"

# 카테고리 (선택지 제한)
class Category(str, Enum):
    TECHNICAL = "기술지원"
    BILLING = "결제문의"
    ACCOUNT = "계정관리"
    GENERAL = "일반문의"

# 지원 티켓 모델
class SupportTicket(BaseModel):
    title: str = Field(description="문제에 대한 간결한 요약 제목")
    description: str = Field(description="문제에 대한 상세한 설명")
    priority: Priority = Field(description="문제의 심각도에 따른 우선순위")
    category: Category = Field(description="티켓이 속하는 부서 또는 카테고리")
    steps_to_reproduce: Optional[List[str]] = Field(
        description="문제를 재현하기 위한 단계별 목록 (기술적 문제인 경우)",
        default=None
    )
    customer_contact: Optional[str] = Field(
        description="고객의 연락처 정보 (제공된 경우)",
        default=None
    )

### 티켓 생성 에이전트 정의

에이전트에게 사용자의 자연어 입력을 분석하여 위에서 정의한 `SupportTicket` 스키마로 변환하도록 지시한다.

In [13]:
ticket_agent = Agent(
    name="ticket_agent",
    model=MODEL,
    description="고객 지원 티켓 생성 어시스턴트",
    instruction="""
    당신은 고객의 문의 내용을 분석하여 정형화된 지원 티켓을 생성하는 AI입니다.
    
    1. 고객의 말에서 핵심 문제를 파악하여 제목과 설명을 작성하세요.
    2. 문제의 심각성을 판단하여 우선순위(Priority)를 결정하세요.
    3. 문의 내용에 맞는 카테고리(Category)를 분류하세요.
    4. 기술적인 문제라면 재현 단계를, 연락처가 있다면 연락처 정보를 추출하세요.
    """,
    output_schema=SupportTicket
)

ticket_runner = InMemoryRunner(agent=ticket_agent)

### 실행 테스트

두 가지 다른 상황을 테스트하여 에이전트가 상황에 따라 필드를 어떻게 채우는지 확인한다.

In [14]:
print("=== 상황 1: 서버 장애 (긴급, 기술) ===")
await ticket_runner.run_debug(
    "어제부터 서버 접속이 안 됩니다. 로그인을 하려고 하면 500 에러가 계속 떠요. "
    "빨리 해결해 주세요! 제 이메일은 user@example.com 입니다."
)

=== 상황 1: 서버 장애 (긴급, 기술) ===

 ### Created new session: debug_session_id

User > 어제부터 서버 접속이 안 됩니다. 로그인을 하려고 하면 500 에러가 계속 떠요. 빨리 해결해 주세요! 제 이메일은 user@example.com 입니다.
ticket_agent > {
  "title": "서버 접속 불가 및 500 에러 발생",
  "description": "어제부터 서버 접속이 불가능하며, 로그인 시 500 에러가 지속적으로 발생합니다. 빠른 해결이 필요합니다.",
  "priority": "긴급",
  "category": "기술지원",
  "steps_to_reproduce": [
    "로그인 시도"
  ],
  "customer_contact": "user@example.com"
}


[Event(model_version='gemini-2.0-flash', content=Content(
   parts=[
     Part(
       text="""{
   "title": "서버 접속 불가 및 500 에러 발생",
   "description": "어제부터 서버 접속이 불가능하며, 로그인 시 500 에러가 지속적으로 발생합니다. 빠른 해결이 필요합니다.",
   "priority": "긴급",
   "category": "기술지원",
   "steps_to_reproduce": [
     "로그인 시도"
   ],
   "customer_contact": "user@example.com"
 }"""
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   candidates_token_count=122,
   candidates_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=122
     ),
   ],
   prompt_token_count=343,
   prompt_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=343
     ),
   ],
   total_token_count=465
 ), live_session_resumpt

In [15]:
print("\n=== 상황 2: 환불 문의 (보통, 결제) ===")
await ticket_runner.run_debug(
    "지난달 요금이 잘못 청구된 것 같아요. 확인 부탁드립니다."
)


=== 상황 2: 환불 문의 (보통, 결제) ===

 ### Continue session: debug_session_id

User > 지난달 요금이 잘못 청구된 것 같아요. 확인 부탁드립니다.
ticket_agent > {
  "title": "지난달 요금 청구 오류 문의",
  "description": "지난달 요금이 잘못 청구된 것으로 예상되어 확인 요청합니다.",
  "priority": "보통",
  "category": "결제문의",
  "steps_to_reproduce": null,
  "customer_contact": null
}


[Event(model_version='gemini-2.0-flash', content=Content(
   parts=[
     Part(
       text="""{
   "title": "지난달 요금 청구 오류 문의",
   "description": "지난달 요금이 잘못 청구된 것으로 예상되어 확인 요청합니다.",
   "priority": "보통",
   "category": "결제문의",
   "steps_to_reproduce": null,
   "customer_contact": null
 }"""
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   candidates_token_count=87,
   candidates_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=87
     ),
   ],
   prompt_token_count=488,
   prompt_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=488
     ),
   ],
   total_token_count=575
 ), live_session_resumption_update=None, input_transcription=None, output_transcription

## 결과 분석 및 정리

출력 로그를 확인해보면 에이전트가 생성한 응답이 JSON 형식으로 깔끔하게 정리되어 있으며, `SupportTicket` 모델의 정의에 따라 `priority`나 `category`가 지정된 값(Enum)으로 정확히 분류된 것을 알 수 있다.

이 기능을 활용하면 AI의 응답을 웹 애플리케이션의 API 응답으로 바로 사용하거나, DB에 저장하는 등 시스템 통합을 매우 쉽게 할 수 있다.

다음 모듈에서는 에이전트가 이전 대화 내용을 기억하고 활용하는 **세션과 메모리(Memory)** 관리에 대해 알아본다.